In [1]:
import arkouda as ak
import arachne as ar

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
ak.connect("n82", 5555)

connected to arkouda server tcp://*:5555


In [3]:
connectome_edges = ak.read("/scratch/users/oaa9/experimentation/data/connectome/h01/edges*")
connectome_edges = ak.DataFrame(connectome_edges)

In [4]:
vals = connectome_edges["confidence"] > 0.75
ak.value_counts(vals)

(array([False True]), array([9405824 140465845]))

In [5]:
connectome_edges["bool_conf"] = vals

In [6]:
connectome_nodes = ak.read("/scratch/users/oaa9/experimentation/data/connectome/h01/nodes*")
connectome_nodes = ak.DataFrame(connectome_nodes)

In [7]:
nodes_gb = connectome_nodes.groupby("neuron_id")
connectome_nodes = connectome_nodes[nodes_gb.permutation[nodes_gb.segments]]

In [8]:
graph = ar.PropGraph()
graph.load_edge_attributes(connectome_edges, source_column="src_neuron_id", destination_column="dst_neuron_id", relationship_columns="type_label")
graph.load_node_attributes(connectome_nodes, node_column="neuron_id")

In [9]:
print(f"Graph has {len(graph):_} vertices and {graph.size():_} edges.")

Graph has 142_660_662 vertices and 147_071_359 edges.


In [10]:
src = ak.array([0, 1, 2, 2])
dst = ak.array([1, 2, 0, 3])
nodes = ak.array([0, 1, 2, 3])

In [11]:
bool_conf = ak.array([True, True, True, True])

In [12]:
subgraph_edge_df = ak.DataFrame({"src":src, "dst":dst, "bool_conf":bool_conf})

In [13]:
subgraph = ar.PropGraph()
subgraph.load_edge_attributes(subgraph_edge_df, source_column="src", destination_column="dst")

In [14]:
def vf2_si(g, h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "si",
                                            reorder_type = "structural", return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [15]:
def vf2_si_probability_reordering(g,h):
    isos_as_vertices = ar.subgraph_isomorphism(g, h, 
                                            semantic_check = "and", algorithm_type = "si",
                                            reorder_type = "probability", return_isos_as = "vertices")
    print(f"We found {len(isos_as_vertices[0])/len(h)} monos inside of the graph")

In [16]:
vf2_si(graph, subgraph)

We found 118225.0 monos inside of the graph


In [17]:
vf2_si_probability_reordering(graph, subgraph)

We found 118225.0 monos inside of the graph
